In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf
import os
import struct
import hashlib
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
import tensorflow_hub as hub
import tokenization
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.cluster import KMeans
import pickle
from sknetwork.ranking import PageRank
from nltk.stem.snowball import EnglishStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [7]:
cnn_dm_train, cnn_dm_val, cnn_dm_test = tfds.load(name="cnn_dailymail", 
                                                  split=["train", "validation", "test"], 
                                                  data_dir="/mnt/disks/disk-1/data")

INFO:absl:Load dataset info from /mnt/disks/disk-1/data/cnn_dailymail/3.1.0
INFO:absl:Reusing dataset cnn_dailymail (/mnt/disks/disk-1/data/cnn_dailymail/3.1.0)
INFO:absl:Constructing tf.data.Dataset cnn_dailymail for split ['train', 'validation', 'test'], from /mnt/disks/disk-1/data/cnn_dailymail/3.1.0
2021-11-28 05:39:26.504796: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-28 05:39:26.995786: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38464 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [ ]:
arxiv_train, arxiv_val, arxiv_test = tfds.load(name="scientific_papers/arxiv", 
                                                  split=["train", "validation", "test"], 
                                                  data_dir="/mnt/disks/disk-1/data")

In [ ]:
pubmed_train, pubmed_val, pubmed_test = tfds.load(name="scientific_papers/pubmed", 
                                                  split=["train", "validation", "test"], 
                                                  data_dir="/mnt/disks/disk-1/data")

In [ ]:
# ds = cnn_dm_train.take(1)
ds = arxiv_train.take(1)
# ds = pubmed_train.take(1)
for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    print(list(example.keys()))
    x = example["article"].numpy().decode('UTF-8').split('\n')
    for l in x:
        print(l)
        print("_")
    print("---")
    x = example["abstract"].numpy().decode('UTF-8').split('\n')
    for l in x:
        print(l)
        print("_")
    break

In [16]:
def tensor_to_string(x):
    return x.numpy().decode('UTF-8')

In [6]:
def get_sent_list(text,stem=None):
    sents = sent_tokenize(text)
    if stem is None:
        return sents
    ans = []
    for sent in sents:
        words = word_tokenize(sent)
        word_stem = [stem.stem(w) for w in words]
        ans.append(" ".join(word_stem))
    return ans

In [8]:
x = list(cnn_dm_test)[0]

2021-11-28 05:39:38.098516: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [19]:
stemmer = EnglishStemmer()
get_sent_list(tensor_to_string(x['highlights']),stem=stemmer)

['expert question if pack out plane are put passeng at risk .',
 'u. consum advisori group say minimum space must be stipul .',
 'safeti test conduct on plane with more leg room than airlin offer .']

In [21]:
def uml_summary(x,kind="cnn_dm",stem=None,model=None):
    if kind == "cnn_dm":
        key1 = 'article'
        key2 = 'highlights'
    elif kind == 'arxiv' or kind == 'pubmed':
        key1 = 'article'
        key2 = 'abstract'
    text = tensor_to_string(x[key1])
    text = get_sent_list(text,stem)
    summary = tensor_to_string(x[key2])
    summary = get_sent_list(summary,stem)
    text_emb = model.encode(text)
    filename = 
    with open('filename.pickle', 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
    gen_sum = [text[x] for x in generate_summary(text_emb,len(summary))]
    scores = scorer.score(" ".join(summary)," ".join(gen_sum))
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

In [23]:
## Model features include an encode function -> takes a list of sentences. Returns a list of embeddings (all same dim)
model = SentenceTransformer('all-MiniLM-L6-v2',cache_folder='/mnt/disks/disk-1/data/models')

transformers = ["all-mpnet-base-v2",
                "multi-qa-mpnet-base-dot-v1",
                "all-distilroberta-v1",
                "all-MiniLM-L12-v2",
                "multi-qa-distilbert-cos-v1",
                "all-MiniLM-L6-v2",
                "multi-qa-MiniLM-L6-cos-v1",
                "paraphrase-multilingual-mpnet-base-v2",
                "paraphrase-albert-small-v2",
                "paraphrase-multilingual-MiniLM-L12-v2",
                "paraphrase-MiniLM-L3-v2",
                "distiluse-base-multilingual-cased-v1",
                "distiluse-base-multilingual-cased-v2"]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)

In [ ]:
## V = list of embeddings. k = target size of summary
## Returns a list of sentence indices

def generate_summary(V, k):
    if k >= len(V):
        return list(range(len(V)))
    k -= 1
    centers = []
    cities = list(range(len(V)))
    centers.append(0)
    cities.remove(0)
    while k!= 0:
        city_dict = {}
        for cty in cities:
            min_dist = float("inf")
            for c in centers:
                min_dist = min(min_dist,np.linalg.norm(V[cty] - V[c]))
            city_dict[cty] = min_dist
        new_center = max(city_dict, key = lambda i: city_dict[i])
        centers.append(new_center)
        cities.remove(new_center)
        k -= 1
    return centers

In [ ]:
## Pagerank version
def sim(a, b):
    return np.dot(a, b) / np.sqrt(np.dot(a, a) * np.dot(b, b))

def generate_summary(V,k):
    if k >= len(V):
        return list(range(len(V)))
    n = V.shape[0]
    adj = np.zeros((n, n))
    for i in range(n):
        adj[i][i] = sim(V[i],V[i])
        for j in range(i+1,n):
            s = sim(V[i], V[j])
            adj[i][j] = s
            adj[j][i] = s

    pr = PageRank()
    scores = pr.fit_transform(adj)
    ind = np.argpartition(scores, -k)[-k:]
    return np.sort(ind)

In [ ]:
## Return R1, R2 and RL score for a text (using the generate_summary function)

def uml_summary(l):
    text,summary = clean(l)
    text_emb = model.encode(text)
    gen_sum = [text[x] for x in generate_summary(text_emb,len(summary))]
    scores = scorer.score(" ".join(summary)," ".join(gen_sum))
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

In [ ]:
%%time
# Don't run this
### all-MiniLM-L6-v2 with K-Centers
r1 = []
r2 = []
rl = []
for i in range(len(url_hashes_val)):
    if i>0 and i%1000 == 0:
        print(i,"Done")
    x = url_hashes_val[i]
    if os.path.exists(os.path.join("CNN-DM","raw","cnn","stories",x+".story")):
        l = open(os.path.join("CNN-DM","raw","cnn","stories",x+".story")).readlines()
    else:
        l = open(os.path.join("CNN-DM","raw","dailymail","stories",x+".story")).readlines()
    r1_val,r2_val,rl_val = uml_summary(l)
    r1.append(r1_val)
    r2.append(r2_val)
    rl.append(rl_val)

# (Uncomment to test for 1 line)
#     break
print("Rouge 1 : ",np.round(np.mean(np.asarray(r1))*100,2))
print("Rouge 2 : ",np.round(np.mean(np.asarray(r2))*100,2))
print("Rouge L : ",np.round(np.mean(np.asarray(rl))*100,2))

In [ ]:
%%time
# Don't run this
### all-MiniLM-L6-v2 with PageRank
r1 = []
r2 = []
rl = []
for i in range(len(url_hashes_val)):
    if i>0 and i%1000 == 0:
        print(i,"Done")
    x = url_hashes_val[i]
    if os.path.exists(os.path.join("CNN-DM","raw","cnn","stories",x+".story")):
        l = open(os.path.join("CNN-DM","raw","cnn","stories",x+".story")).readlines()
    else:
        l = open(os.path.join("CNN-DM","raw","dailymail","stories",x+".story")).readlines()
    r1_val,r2_val,rl_val = uml_summary(l)
    r1.append(r1_val)
    r2.append(r2_val)
    rl.append(rl_val)

# (Uncomment to test for 1 line)
#     break
print("Rouge 1 : ",np.round(np.mean(np.asarray(r1))*100,2))
print("Rouge 2 : ",np.round(np.mean(np.asarray(r2))*100,2))
print("Rouge L : ",np.round(np.mean(np.asarray(rl))*100,2))

In [ ]:
print("".join(l))

In [ ]:
text,summary = clean(l)
print("\n".join(text))
print("___")
print("\n".join(summary))

In [ ]:
text